<a href="https://colab.research.google.com/github/MrZuberi/Lung-Cancer-Diagnostic-Tool/blob/main/lungcancerdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
upload_file = files.upload()

Saving Lung Cancer.csv to Lung Cancer.csv


# Task
Train and produce a usable CNN model using the "Lung Cancer.csv" dataset.

## Data loading and exploration

### Subtask:
Load the "Lung Cancer.csv" file into a pandas DataFrame and explore the dataset to understand its structure, features, and potential issues.


**Reasoning**:
The first step is to load the data and explore its basic structure and statistics. This involves importing pandas, reading the CSV file, displaying the head, info, and descriptive statistics.



In [3]:
import pandas as pd

df = pd.read_csv('Lung Cancer.csv')

# Display the first 5 rows
display(df.head())

# Display column names and data types
display(df.info())

# Display descriptive statistics for numerical columns
display(df.describe())

,id,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,64.0,Male,Sweden,2016-04-05,Stage I,Yes,Passive Smoker,29.4,199,0,0,1,0,Chemotherapy,2017-09-10,0
1,2,50.0,Female,Netherlands,2023-04-20,Stage III,Yes,Passive Smoker,41.2,280,1,1,0,0,Surgery,2024-06-17,1
2,3,65.0,Female,Hungary,2023-04-05,Stage III,Yes,Former Smoker,44.0,268,1,1,0,0,Combined,2024-04-09,0
3,4,51.0,Female,Belgium,2016-02-05,Stage I,No,Passive Smoker,43.0,241,1,1,0,0,Chemotherapy,2017-04-23,0
4,5,37.0,Male,Luxembourg,2023-11-29,Stage I,No,Passive Smoker,19.7,178,0,0,0,0,Combined,2025-01-08,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890000 entries, 0 to 889999
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  890000 non-null  int64  
 1   age                 890000 non-null  float64
 2   gender              890000 non-null  object 
 3   country             890000 non-null  object 
 4   diagnosis_date      890000 non-null  object 
 5   cancer_stage        890000 non-null  object 
 6   family_history      890000 non-null  object 
 7   smoking_status      890000 non-null  object 
 8   bmi                 890000 non-null  float64
 9   cholesterol_level   890000 non-null  int64  
 10  hypertension        890000 non-null  int64  
 11  asthma              890000 non-null  int64  
 12  cirrhosis           890000 non-null  int64  
 13  other_cancer        890000 non-null  int64  
 14  treatment_type      890000 non-null  object 
 15  end_treatment_date  890000 non-nul

None

,id,age,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,survived
count,890000.000000,890000.000000,890000.000000,890000.000000,890000.000000,890000.000000,890000.000000,890000.000000,890000.000000
mean,445000.500000,55.007008,30.494172,233.633916,0.750024,0.469740,0.225956,0.088157,0.220229
std,256921.014128,9.994485,8.368539,43.432278,0.432999,0.499084,0.418211,0.283524,0.414401
min,1.000000,4.000000,16.000000,150.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,222500.750000,48.000000,23.300000,196.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,445000.500000,55.000000,30.500000,242.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,667500.250000,62.000000,37.700000,271.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,890000.000000,104.000000,45.000000,300.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Data preprocessing

### Subtask:
Handle missing values, encode categorical features, scale numerical features, and prepare the data for input into the CNN model. This may involve creating image-like representations of the tabular data if necessary.


**Reasoning**:
Examine the unique values in the 'object' type columns to understand their content and guide the encoding process, and extract relevant features from the date columns.



In [4]:
for col in ['gender', 'country', 'cancer_stage', 'family_history', 'smoking_status', 'treatment_type']:
    display(f"Unique values for column '{col}': {df[col].unique()[:20]}")

# Convert date columns to datetime objects
df['diagnosis_date'] = pd.to_datetime(df['diagnosis_date'])
df['end_treatment_date'] = pd.to_datetime(df['end_treatment_date'])

# Calculate treatment duration
df['treatment_duration'] = (df['end_treatment_date'] - df['diagnosis_date']).dt.days

# Extract year and month from diagnosis_date
df['diagnosis_year'] = df['diagnosis_date'].dt.year
df['diagnosis_month'] = df['diagnosis_date'].dt.month

# Drop original date columns
df = df.drop(columns=['diagnosis_date', 'end_treatment_date'])

display(df.head())

"Unique values for column 'gender': ['Male' 'Female']"

"Unique values for column 'country': ['Sweden' 'Netherlands' 'Hungary' 'Belgium' 'Luxembourg' 'Italy' 'Croatia'\n 'Denmark' 'Malta' 'Germany' 'Poland' 'Ireland' 'Romania' 'Spain' 'Greece'\n 'Estonia' 'Cyprus' 'France' 'Slovenia' 'Latvia']"

"Unique values for column 'cancer_stage': ['Stage I' 'Stage III' 'Stage IV' 'Stage II']"

"Unique values for column 'family_history': ['Yes' 'No']"

"Unique values for column 'smoking_status': ['Passive Smoker' 'Former Smoker' 'Never Smoked' 'Current Smoker']"

"Unique values for column 'treatment_type': ['Chemotherapy' 'Surgery' 'Combined' 'Radiation']"

,id,age,gender,country,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,survived,treatment_duration,diagnosis_year,diagnosis_month
0,1,64.0,Male,Sweden,Stage I,Yes,Passive Smoker,29.4,199,0,0,1,0,Chemotherapy,0,523,2016,4
1,2,50.0,Female,Netherlands,Stage III,Yes,Passive Smoker,41.2,280,1,1,0,0,Surgery,1,424,2023,4
2,3,65.0,Female,Hungary,Stage III,Yes,Former Smoker,44.0,268,1,1,0,0,Combined,0,370,2023,4
3,4,51.0,Female,Belgium,Stage I,No,Passive Smoker,43.0,241,1,1,0,0,Chemotherapy,0,443,2016,2
4,5,37.0,Male,Luxembourg,Stage I,No,Passive Smoker,19.7,178,0,0,0,0,Combined,0,406,2023,11


**Reasoning**:
Encode the categorical features using one-hot encoding for nominal variables and ordinal encoding for the 'cancer_stage' column, and scale the numerical features using StandardScaler.



In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Define categorical and numerical features
categorical_features = ['gender', 'country', 'family_history', 'smoking_status', 'treatment_type']
# 'cancer_stage' is ordinal, will be handled separately
numerical_features = ['age', 'bmi', 'cholesterol_level', 'treatment_duration', 'diagnosis_year', 'diagnosis_month', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer']

# Create an ordinal encoder for 'cancer_stage'
cancer_stage_order = ['Stage I', 'Stage II', 'Stage III', 'Stage IV']
df['cancer_stage_encoded'] = df['cancer_stage'].map({stage: i for i, stage in enumerate(cancer_stage_order)})

# Create a column transformer for one-hot encoding and scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('scaler', StandardScaler(), numerical_features)
    ],
    remainder='passthrough' # Keep other columns (like 'id', 'survived', 'cancer_stage_encoded')
)

# Apply the preprocessing
preprocessed_data = preprocessor.fit_transform(df)

# Convert the preprocessed data back to a DataFrame (optional, but helpful for inspection)
# Get the new column names after one-hot encoding
onehot_feature_names = preprocessor.named_transformers_['onehot'].get_feature_names_out(categorical_features)
# Get the names of the numerical features
numerical_feature_names = numerical_features
# Get the names of the remaining columns (excluding the original categorical and numerical ones)
remaining_features = [col for col in df.columns if col not in categorical_features + numerical_features + ['cancer_stage']]


all_feature_names = list(onehot_feature_names) + numerical_feature_names + remaining_features

preprocessed_df = pd.DataFrame(preprocessed_data, columns=all_feature_names)

display(preprocessed_df.head())